# What is AIND?

In Desbordante we consider an approximate inclusion dependency (AIND)
as any inclusion dependency (IND) that utilizes an error metric to measure
violations.

This metric calculates the proportion of distinct values in the
dependent set (LHS) that must be removed to satisfy the dependency on the
referenced set (RHS) completely.

The metric lies within the `[0, 1]` range:
- A value of `0` means the IND holds exactly (no violations exist).
- A value closer to `1` indicates a significant proportion of LHS values violate the dependency.

## What you can do with it?

Desbordante supports the **discovery** and **verification** of both exact INDs and approximate INDs:
  1. Exact INDs: All values in the LHS set must match a value in the RHS set.
  2. Approximate INDs: Allows for controlled violations quantified by the error metric.

For `discovery` tasks, users can specify an error threshold, and Desbordante will return all AINDs with an error value equal to or less than the specified threshold.

For `verification` tasks, users can specify an AIND, and Desbordante will calculate the error value, identifying clusters of violating values.

The error metric used for AINDs is an adaptation of `g3`, originally designed for approximate functional dependencies (FDs).

For more information, consider:

```
"Unary and n-ary inclusion dependency discovery in relational databases",
        Fabien De Marchi, Stéphane Lopes, and Jean-Marc Petit.
```

# Demonstration

We will show how you can discover and verify both exact and approximate inlusion dependencies.

# Install python dependencies

In [1]:
!pip install desbordante==2.3.2
!pip install pandas
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 20.2 MB/s eta 0:00:00


# Import python modules

In [17]:
import desbordante
import pandas as pd
from tabulate import tabulate
import textwrap

# Get sample datasets

In [23]:
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/employee.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/project_assignments.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/course.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/department.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/instructor.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/student.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/teaches.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/orders.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/products.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/ind_datasets/customers.csv

# AIND's discovery

# Examine the data

The datasets under consideration for this example are `employee` and `project_assignments`.

In [ ]:
pd.read_csv("employee.csv", header=[0])

,id,name,department,location
0,101,Alice Cooper,Marketing,New York
1,102,Bob Johnson,Engineering,San Francisco
2,103,Charlie Brown,HR,Chicago
3,104,Dana White,Sales,Los Angeles
4,105,Eva Black,Marketing,Boston
5,106,Frank Green,Engineering,Austin


In [ ]:
pd.read_csv("project_assignments.csv", header=[0])

,id,employee_name,title,deadline
0,P001,Alice Cooper,Website Redesign,2024-12-01
1,P002,Bob Johnson,App Development,2024-12-15
2,P003,Charley Brown,HR Policy Update,2024-12-20
3,P006,Frank Green,Infrastructure Upgrade,2025-02-05


Let's find all AINDs with an error threshold less than `0.3`.

In [ ]:
algo = desbordante.ind.algorithms.Mind()

TABLES = [
    ("employee.csv", ',', True),
    ("project_assignments.csv", ",", True),
]

algo.load_data(tables=TABLES)
algo.execute(error=0.3)

for ind in algo.get_inds():
    print("IND:", ind)

IND: (project_assignments.csv, [employee_name]) -> (employee.csv, [name]) with error threshold = 0.25


We found only a single AIND, this dependency contains typos in the `employee name` column of the `project_assignment.csv`.

For automatically detecting violating clusters, you can create a pipeline using the AIND verifier in combination with a mining algorithm.

## Exact IND's discovery

Consider the following data tables.

In [8]:
pd.read_csv("course.csv")

,Course ID,Title,Department name
0,IT-1,Computer Science,Institute of Information Technology
1,MM-3,Algebra,Mathematics and Mechanics Faculty
2,H-1,History,Institute of History
3,FL-2,English,Faculty of Foreign Languages
4,IT-2,Programming,Institute of Information Technology
5,S-5,Philosophy,Faculty of Sociology
6,P-2,Physics,Faculty of Physics
7,C-8,Chemistry,Institute of Chemistry


In [9]:
pd.read_csv("department.csv")

,Department name,Building
0,Institute of Information Technology,5 Academic av.
1,Mathematics and Mechanics Faculty,3 Academic av.
2,Institute of History,29A University st.
3,Faculty of Foreign Languages,10 Science sq.
4,Faculty of Sociology,29C University st.
5,Faculty of Physics,10 Academic av.
6,Institute of Chemistry,11 Academic av.
7,Graduate School of Managemment,49 Science sq.


In [10]:
pd.read_csv("instructor.csv")

,ID,Name,Department name,Salary
0,in1089,Prof. Jones,Mathematics and Mechanics Faculty,$12000
1,in6723,Dr. Powers,Faculty of Sociology,$8000
2,in5555,Larry Thompson,Graduate School of Managemment,$5000
3,in8930,Prof. Burgess,Faculty of Sociology,$11500
4,in4520,David Stewart,Institute of Chemistry,$5200
5,in6577,Dr. Holloway,Mathematics and Mechanics Faculty,$9000
6,in9910,Dr. Rose,Institute of History,$8500


In [11]:
pd.read_csv("student.csv")

,ID,Name,Department name
0,st104726,Darlene Johnson,Institute of Chemistry
1,st967925,Alice Green,Mathematics and Mechanics Faculty
2,st760375,Olga Jones,Graduate School of Managemment
3,st779090,Felix Brown,Faculty of Sociology
4,st299471,Angela Ramirez,Faculty of Sociology
5,st887788,Debbie Lewis,Graduate School of Managemment
6,st679973,Evelyn Obrien,Mathematics and Mechanics Faculty
7,st897856,Melissa Smith,Institute of Information Technology


In [12]:
pd.read_csv("teaches.csv")

,Instructor ID,Course ID,Year,Semester
0,in1089,MM-3,2,Fall
1,in6723,S-5,1,Spring
2,in8930,S-5,3,Fall
3,in4520,C-8,2,Fall
4,in6577,MM-3,1,Fall


Let's discover exact AIND's of the given tables together.

`->` means "is included in"

In [15]:
TABLES = [(f'{table_name}.csv', ',', True) for table_name in ['course', 'department', 'instructor', 'student', 'teaches']]

algo = desbordante.ind.algorithms.Default()
algo.load_data(tables=TABLES)
algo.execute()
inds = algo.get_inds()

In [16]:
for ind in inds:
    print(ind)

(course.csv, [Department name]) -> (department.csv, [Department name])
(instructor.csv, [Department name]) -> (department.csv, [Department name])
(student.csv, [Department name]) -> (department.csv, [Department name])
(teaches.csv, [Instructor ID]) -> (instructor.csv, [ID])
(teaches.csv, [Course ID]) -> (course.csv, [Course ID])


## Verifying exact INDs

Let's start with the exact IND verification scenario.

The datasets under consideration for this scenario are `orders`
and `products`.
Let's start by verifying exact IND holding between those tables.

In [21]:
def get_table_df(dataset):
    return pd.read_csv(f"{dataset}.csv", header=[0])

def aind_str(lhs, rhs):
    def cc_str(cc):
        (table_name, df, indices) = cc
        columns = [df.columns[idx] for idx in indices]
        return ", ".join(f"{table_name}.{col}" for col in columns)

    return f"[{cc_str(lhs)}] -> [{cc_str(rhs)}]"

def print_table(table, title):
    print(title)
    print(tabulate(table, headers='keys', tablefmt='psql'), end='\n\n')

def print_results_for_ind(verifier):
    if verifier.get_error() == 0:
        print("IND holds")
    else:
        print(f"AIND holds with error = {verifier.get_error():.2}")

def aind_verify(lhs, rhs):
    (lhs_table_name, lhs_table, lhs_indices) = lhs
    (rhs_table_name, rhs_table, rhs_indices) = rhs

    print_table(lhs_table, f"Dataset '{lhs_table_name}':")
    print_table(rhs_table, f"Dataset '{rhs_table_name}':")

    print(f"Checking the IND {aind_str((lhs_table_name, lhs_table, lhs_indices), (rhs_table_name, rhs_table, rhs_indices))}")

    algo = desbordante.aind_verification.algorithms.Default()
    algo.load_data(tables=[lhs_table, rhs_table])
    algo.execute(lhs_indices=lhs_indices, rhs_indices=rhs_indices)

    return algo


algo = aind_verify(('orders', get_table_df('orders'), [2]),
                    ('products', get_table_df('products'), [1]))

print_results_for_ind(algo)

Dataset 'orders':
+----+------+---------------+-----------+
|    |   id |   customer_id | product   |
|----+------+---------------+-----------|
|  0 |    1 |           101 | Laptop    |
|  1 |    2 |           102 | Phone     |
|  2 |    3 |           103 | Tablet    |
|  3 |    4 |           104 | Monitor   |
|  4 |    5 |           108 | Keyboard  |
|  5 |    6 |           201 | Mouse     |
|  6 |    7 |           102 | Charger   |
+----+------+---------------+-----------+

Dataset 'products':
+----+------+----------+-------------+
|    |   id | name     | category    |
|----+------+----------+-------------|
|  0 |    1 | Laptop   | Electronics |
|  1 |    2 | Phone    | Electronics |
|  2 |    3 | Tablet   | Electronics |
|  3 |    4 | Monitor  | Electronics |
|  4 |    5 | Keyboard | Accessories |
|  5 |    6 | Mouse    | Accessories |
|  6 |    7 | Charger  | Accessories |
+----+------+----------+-------------+

Checking the IND [orders.product] -> [products.name]
IND holds


The IND holds because there are no inconsistencies between the two tables. The `products.name` column acts as a primary key, and all values in the `orders.product` column match entries in `products.name` without any typos or missing data.

# Verifying Approximate INDs

Now, let's consider the approximate IND verification scenario (AIND).
Unlike exact INDs, approximate INDs allow for a certain level of error.
This error indicates how accurately the dependency holds between the datasets.

In this scenario, we will use the `orders` and `customers` datasets.

In [24]:
verify = lambda lhs_df, rhs_df: aind_verify(('orders', lhs_df, [1]),
                                            ('customers', rhs_df, [0]))

lhs_df = get_table_df('orders')
rhs_df = get_table_df('customers')

algo = verify(lhs_df, rhs_df)

print_results_for_ind(algo)

Dataset 'orders':
+----+------+---------------+-----------+
|    |   id |   customer_id | product   |
|----+------+---------------+-----------|
|  0 |    1 |           101 | Laptop    |
|  1 |    2 |           102 | Phone     |
|  2 |    3 |           103 | Tablet    |
|  3 |    4 |           104 | Monitor   |
|  4 |    5 |           108 | Keyboard  |
|  5 |    6 |           201 | Mouse     |
|  6 |    7 |           102 | Charger   |
+----+------+---------------+-----------+

Dataset 'customers':
+----+------+---------+-----------+
|    |   id | name    | country   |
|----+------+---------+-----------|
|  0 |  101 | Alice   | USA       |
|  1 |  102 | Bob     | UK        |
|  2 |  103 | Charlie | Canada    |
|  3 |  104 | David   | Germany   |
|  4 |  105 | Eve     | France    |
+----+------+---------+-----------+

Checking the IND [orders.customer_id] -> [customers.id]
AIND holds with error = 0.33


We see that this AIND has an error of `0.33`.
Let's examine the violating clusters in more detail to understand the errors.

In [27]:
def print_clusters(verifier, lhs):
    (table_name, table, indices) = lhs

    print(f"Number of clusters violating IND: {verifier.get_violating_clusters_count()}")
    for i, cluster in enumerate(verifier.get_violating_clusters(), start=1):
        print(f"#{i} cluster:")
        for el in cluster:
            values = " ".join([f"{table[table.columns[idx]][el]}" for idx in indices])
            print(f"\t{el}: {values}")

print_clusters(algo, ('orders', lhs_df, [1]))

Number of clusters violating IND: 2
#1 cluster:
	5: 201
#2 cluster:
	4: 108


Based on our analysis, this AIND does not hold due to the following reasons:
1. The `orders.customer_id` value `201` does not match any entry in the `customers.id` column. This suggests a possible typo where `201` might have been entered instead of `101`, indicating that the customer who bought the `Mouse` should be `Alice`.
2. The `orders.customer_id` value `108` also violates the AIND. This appears to be a case where the `customers` table might be incomplete,
and some customer entries are missing.

In such cases, resolving typos and ensuring data completeness in the reference table `customers` can help improve the accuracy of this dependency.

Let's fix the issues.
1. Fix data issue in the `orders` dataset by updating the value in the `orders.customer_id` column where it is `201` to `101`.

In [28]:
lhs_df.loc[lhs_df['customer_id'] == 201, 'customer_id'] = 101
algo = verify(lhs_df, rhs_df)

print_results_for_ind(algo)

Dataset 'orders':
+----+------+---------------+-----------+
|    |   id |   customer_id | product   |
|----+------+---------------+-----------|
|  0 |    1 |           101 | Laptop    |
|  1 |    2 |           102 | Phone     |
|  2 |    3 |           103 | Tablet    |
|  3 |    4 |           104 | Monitor   |
|  4 |    5 |           108 | Keyboard  |
|  5 |    6 |           101 | Mouse     |
|  6 |    7 |           102 | Charger   |
+----+------+---------------+-----------+

Dataset 'customers':
+----+------+---------+-----------+
|    |   id | name    | country   |
|----+------+---------+-----------|
|  0 |  101 | Alice   | USA       |
|  1 |  102 | Bob     | UK        |
|  2 |  103 | Charlie | Canada    |
|  3 |  104 | David   | Germany   |
|  4 |  105 | Eve     | France    |
+----+------+---------+-----------+

Checking the IND [orders.customer_id] -> [customers.id]
AIND holds with error = 0.2


We have successfully fixed the typo in the `orders` dataset.
Now, let's address the missing customer entry.


2. Add the missing customer to the `customers` dataset.
Adding a new customer with id `108`, name `Frank`, and country `Italy.

In [29]:
new_entry = {'id': 108, 'name': 'Frank', 'country': 'Italy'}
rhs_df = pd.concat([rhs_df, pd.DataFrame([new_entry])], ignore_index=True)
algo = verify(lhs_df, rhs_df)

print_results_for_ind(algo)

Dataset 'orders':
+----+------+---------------+-----------+
|    |   id |   customer_id | product   |
|----+------+---------------+-----------|
|  0 |    1 |           101 | Laptop    |
|  1 |    2 |           102 | Phone     |
|  2 |    3 |           103 | Tablet    |
|  3 |    4 |           104 | Monitor   |
|  4 |    5 |           108 | Keyboard  |
|  5 |    6 |           101 | Mouse     |
|  6 |    7 |           102 | Charger   |
+----+------+---------------+-----------+

Dataset 'customers':
+----+------+---------+-----------+
|    |   id | name    | country   |
|----+------+---------+-----------|
|  0 |  101 | Alice   | USA       |
|  1 |  102 | Bob     | UK        |
|  2 |  103 | Charlie | Canada    |
|  3 |  104 | David   | Germany   |
|  4 |  105 | Eve     | France    |
|  5 |  108 | Frank   | Italy     |
+----+------+---------+-----------+

Checking the IND [orders.customer_id] -> [customers.id]
IND holds


The missing customer has been successfully added to the `customers` dataset.

All issues in the `orders` and `customers` datasets have been resolved!